In [46]:
from nuscenes.nuscenes import NuScenes
from nuscenes.utils import splits
import os
from pathlib import Path
from tqdm import tqdm
from pyquaternion import Quaternion



In [ ]:
nusc = NuScenes(version='v1.0-mini',
                dataroot='../data/nuscenes/mini',
                verbose=True)


In [57]:
# nusc.list_scenes()
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)
# nusc.list_sample(my_sample['token'])
camera_front_data = nusc.get('sample_data', my_sample['data']['CAM_FRONT'])
lidar_top = nusc.get('sample_data', my_sample['data']['LIDAR_TOP'])
a, b, c = nusc.get_sample_data(my_sample['data']['LIDAR_TOP'])
pose_record = nusc.get("ego_pose", lidar_top['ego_pose_token'])
cs = nusc.get('calibrated_sensor', lidar_top['calibrated_sensor_token'])
cs = nusc.get('calibrated_sensor', camera_front_data['calibrated_sensor_token'])

# my_scene,
# first_sample_token,
# my_sample,
#  my_sample['data'], 
# camera_front_data
# lidar_top
# b
# pose_record
cs
# nusc.get_sample_data( my_sample['data']['CAM_FRONT'])[2]
nusc.render_sample_data()

{'token': '1d31c729b073425e8e0202c5c6e66ee1',
 'sensor_token': '725903f5b62f56118f4094b46a4470d8',
 'translation': [1.70079118954, 0.0159456324149, 1.51095763913],
 'rotation': [0.4998015430569128,
  -0.5030316162024876,
  0.4997798114386805,
  -0.49737083824542755],
 'camera_intrinsic': [[1266.417203046554, 0.0, 816.2670197447984],
  [0.0, 1266.417203046554, 491.50706579294757],
  [0.0, 0.0, 1.0]]}

In [38]:
def is_filepath(x):
    return isinstance(x, Path) or isinstance(x, str)


def check_file_exist(filename, msg_tmpl='file "{}" does not exist'):
    if not os.path.isfile(filename):
        raise FileNotFoundError(msg_tmpl.format(filename))


def get_available_scenes(nusc: NuScenes):
    """Get avaliable scenes from nuscenes class"""
    available_scenes = []
    print("Total scene num: {}".format(len(nusc.scene)))
    for scene in nusc.scene:
        scene_token = scene['token']
        scene_rec = nusc.get('scene', scene_token)
        # sample means current frame.
        sample_rec = nusc.get('sample', scene_rec['first_sample_token'])
        sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])
        has_more_frames = True
        scene_not_exist = False
        while has_more_frames:
            lidar_path, boxes, _ = nusc.get_sample_data(sd_rec['token'])
            lidar_path = str(lidar_path)
            if os.getcwd() in lidar_path:
                # path from lyftdataset is absolute path
                lidar_path = lidar_path.split(f'{os.getcwd()}/')[-1]
                # relative path
            if not is_filepath(lidar_path):
                scene_not_exist = True
                break
            else:
                break
        if scene_not_exist:
            continue
        available_scenes.append(scene)
    print('Exist scene num: {}'.format(len(available_scenes)))
    return available_scenes


def _fill_trainval_info(nusc, train_scenes, val_scenes, max_sweeps=10):
    """Generate the train/val infos from the raw data"""
    train_nusc_infos = {}
    val_nusc_infos = {}
    frame_idx = 0
    for sample in tqdm(nusc.sample):
        lidar_token = sample['data']['LIDAR_TOP']
        sd_rec = nusc.get('sample_data', lidar_token)
        cs_record = nusc.get("calibrated_sensor",
                             sd_rec['calibrated_sensor_token'])
        pose_record = nusc.get('ego_pose', sd_rec['ego_pose_token'])
        lidar_path, boxes, _ = nusc.get_sample_data(lidar_token)

        # check file exist.
        check_file_exist(lidar_path)

        can_bus = []

        # init info
        info = {
            'lidar_path': lidar_path,
            'frame_token': sample['token'],
            'frame_prev': sample['prev'],
            'frame_next': sample['next'],
            'can_bus': can_bus,
            'frame_index': frame_idx,
            'sweeps': [],
            'cams': dict(),
            'scene_token': sample['scene_token'],
            'lidar2ego_translation': cs_record['translation'],
            'lidar2ego_rotation': cs_record['rotation'],
            'ego2global_translation': pose_record['translation'],
            'ego2global_rotation': pose_record['rotation'],
            'timestamp': sample['timestamp'],
        }

        # no next frame
        if sample['next'] == '':
            frame_idx = 0
        else:
            frame_idx += 1

        # get coord-trans info
        l2e_r = info['lidar2ego_rotation']
        l2e_t = info['lidar2ego_translation']
        e2g_r = info['ego2global_rotation']
        e2g_t = info['ego2global_translation']
        l2e_r_mat = Quaternion(l2e_r).rotation_matrix
        e2g_r_mat = Quaternion(e2g_r).rotation_matrix

        # obtain 6 images' information per frame
        camera_type = [
            'CAM_FRONT',
            'CAM_FRONT_RIGHT',
            'CAM_FRONT_LEFT',
            'CAM_BACK',
            'CAM_BACK_LEFT',
            'CAM_BACK_RIGHT',
        ]

        for camera in camera_type:
            camera_token = sample['data'][camera]
            _, _, cam_intrinsic = nusc.get_sample_data(camera_token)
            

version = 'v1.0-mini'
if version == 'v1.0-trainval':
    train_scenes = splits.train
    val_scenes = splits.val
elif version == 'v1.0-test':
    train_scenes = splits.test
    val_scenes = []
elif version == 'v1.0-mini':
    train_scenes = splits.mini_train
    val_scenes = splits.mini_val
else:
    raise ValueError('unknown')
available_scenes = get_available_scenes(nusc)
available_scene_names = [s['name'] for s in available_scenes]
train_scenes = list(filter(lambda x: x in available_scene_names, train_scenes))
val_scenes = list(filter(lambda x: x in available_scene_names, val_scenes))
train_scenes = set([
    available_scenes[available_scene_names.index(s)]['token']
    for s in train_scenes
])
val_scenes = set([
    available_scenes[available_scene_names.index(s)]['token']
    for s in val_scenes
])
print('train scene: {}, val scene: {}'.format(len(train_scenes),
                                              len(val_scenes)))


Total scene num: 10
Exist scene num: 10
train scene: 8, val scene: 2


In [41]:
nusc.sample

[{'token': 'ca9a282c9e77460f8360f564131a8af5',
  'timestamp': 1532402927647951,
  'prev': '',
  'next': '39586f9d59004284a7114a68825e8eec',
  'scene_token': 'cc8c0bf57f984915a77078b10eb33198',
  'data': {'RADAR_FRONT': '37091c75b9704e0daa829ba56dfa0906',
   'RADAR_FRONT_LEFT': '11946c1461d14016a322916157da3c7d',
   'RADAR_FRONT_RIGHT': '491209956ee3435a9ec173dad3aaf58b',
   'RADAR_BACK_LEFT': '312aa38d0e3e4f01b3124c523e6f9776',
   'RADAR_BACK_RIGHT': '07b30d5eb6104e79be58eadf94382bc1',
   'LIDAR_TOP': '9d9bf11fb0e144c8b446d54a8a00184f',
   'CAM_FRONT': 'e3d495d4ac534d54b321f50006683844',
   'CAM_FRONT_RIGHT': 'aac7867ebf4f446395d29fbd60b63b3b',
   'CAM_BACK_RIGHT': '79dbb4460a6b40f49f9c150cb118247e',
   'CAM_BACK': '03bea5763f0f4722933508d5999c5fd8',
   'CAM_BACK_LEFT': '43893a033f9c46d4a51b5e08a67a1eb7',
   'CAM_FRONT_LEFT': 'fe5422747a7d4268a4b07fc396707b23'},
  'anns': ['ef63a697930c4b20a6b9791f423351da',
   '6b89da9bf1f84fd6a5fbe1c3b236f809',
   '924ee6ac1fed440a9d9e3720aac635a0',
